# MQ-9

## Specifications

### Standard Work Condition

    Vc    (Circuit Voltage)        :  5v +/- 0.1
    VH(h) (Heating Voltage - high) :  5v +/- 0.1
    VH(l) (Heating Voltage - low)  :  1.4v +/- 0.1
    RL    (Load Resistance)        :  can adjust
    RH    (Heater Resistance)      :  33 Ohm +/- 5%
    TH(H) (Heating time (high)     :  60 +/- 1 seconds
    TH(L) (Heating time (low)      :  90 +/- 1 seconds
    PH    (Heating consumption)    :  less than 340 mW
    

### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 2kOhm  -  20kOhm (100ppm Carbon Monoxide)
    Preheat Time               : No less than 48 hours
    
    Detecting range:
        Carbon Monoxide :  20ppm  - 2000ppm 
        CH4             :  500ppm - 10000ppm 
        LPG             :  500ppm - 10000ppm 


In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Liquefied Petroleum Gas
lpg = {
    'p1' : {'x': 50,   'y': 2.092},
    'p2' : {'x': 500,  'y': 1.361},
    'p3' : {'x': 800,  'y': 1.108},
    'p4' : {'x': 1000, 'y': 1.000},
    'p5' : {'x': 1500, 'y': 0.824},
    'p6' : {'x': 2000, 'y': 0.726},
    'p7' : {'x': 3000, 'y': 0.594},
    'p8' : {'x': 5000, 'y': 0.464},
    'p8' : {'x': 10000,'y': 0.332}
}

# Carbon Monoxide
co = {
    'p1' : {'x': 50,   'y': 1.631},
    'p2' : {'x': 500,  'y': 1.068},
    'p3' : {'x': 800,  'y': 0.869},
    'p4' : {'x': 1000, 'y': 0.785}
}

# Methane
ch4 = {
    'p1' : {'x': 50,   'y': 3.122},
    'p2' : {'x': 500,  'y': 2.294},
    'p3' : {'x': 800,  'y': 1.913},
    'p4' : {'x': 1000, 'y': 1.769},
    'p5' : {'x': 1500, 'y': 1.539},
    'p6' : {'x': 2000, 'y': 1.372},
    'p7' : {'x': 3000, 'y': 1.173},
    'p8' : {'x': 5000, 'y': 0.949},
    'p8' : {'x': 10000,'y': 0.701}
}


AIR = 9.774

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  y, m, b: 10 ** ((math.log10(y) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
## Test the _getPoints_ function
"""
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']
_lpg  = [ lpg[n]      for n in list(lpg)      if n in points]


racio  = 1
p1, p2  = getPoints(racio, _lpg)

m = M(p1,p2)
b = B(p1, m)

lpg_ppm  = PPM(racio, m, b)
print(f"{p1}\n{p2}")
print(f"[ m: {m:.2f} || b: {b:.2f} ]")
print(f"{lpg_ppm:.0f}ppm")
"""
print()

In [6]:
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']

_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]

#print(_lpg)
#print(_co)
#print(_ch4)

In [7]:
_ratios   = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

lpg_ppm     = []
co_ppm      = []
ch4_ppm     = []

for r in _ratios:
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
print ("{:<8} {:<13} {:<13} {:<13}"
       .format('Ratio','LPG', 'CO', 'CH4'))
for i in range(len(_ratios)):
    print ("{:<8} {:<13} {:<13} {:<13}"
           .format( _ratios[i],
                   round(lpg_ppm[i],3),
                   round(co_ppm[i],3),
                   round(ch4_ppm[i],3),
                  ))

Ratio    LPG           CO            CH4          
10       0.011         0.003         0.008        
9        0.02          0.005         0.018        
8        0.038         0.009         0.044        
7        0.078         0.018         0.12         
6        0.177         0.042         0.379        
5        0.47          0.113         1.482        
4        1.553         0.38          7.849        
3        7.251         1.818         67.347       
2        63.618        16.492        713.029      
1        1000.0        580.89        4356.985     
0.9      1246.923      738.57        5574.393     
0.8      1604.196      959.304       7342.199     
0.7      2152.944      1286.025      10033.441    
0.6      2939.693      1803.841      14388.517    
0.5      4285.097      2691.547      22039.131    
0.4      6800.248      4392.589      37139.371    
0.3      12333.798     8259.666      72782.072    
0.2      28545.24      20113.367     187863.715   
0.1      119823.673    92097.75

In [8]:
# MQ 9 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.06   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR   

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 9 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.25   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))
    

Sensor   Rs       Ro       Ratio   
1        823.33   84.24    9.77    
2        190.0    19.44    9.77    


In [9]:
## PPM

sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])

sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])

####

sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])

sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])

sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])

####

print ("{:<8} {:<8} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','LPG', 'CO', 'CH4'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),3),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),3),
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),3),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),3),
          ))

Volt     Ratio    LPG           CO            CH4          
0.06     9.77     0.013         0.003         0.01         
0.25     9.77     0.013         0.003         0.01         


In [10]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','LPG', 'CO', 'CH4'))

for idx in range(len(projection['ratio'])):
    r = projection['ratio'][idx]
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
               round(PPM(r, projection['co']['m'], projection['co']['b']),2),
               round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
              ))
    

    Volt     Ratio    LPG           CO            CH4          
0   0.06     9.77     0.01          0.0           0.01         
1   0.16     3.59     2.77          0.68          17.57        
2   0.26     2.16     41.69         10.74         581.33       
3   0.36     1.53     267.05        70.77         1522.02      
4   0.46     1.17     704.14        302.15        3008.14      
5   0.56     0.94     1135.28       666.9         5020.1       
6   0.66     0.78     1696.09       1012.34       7775.48      
7   0.76     0.66     2407.83       1452.24       11420.7      
8   0.86     0.57     3249.87       2007.38       16124.46     
9   0.96     0.5      4292.52       2696.49       22082.28     
10  1.06     0.44     5550.14       3541.23       29521.9      
11  1.16     0.39     7054.08       4566.68       38709.64     
12  1.26     0.35     8840.59       5802.07       49958.2      
13  1.36     0.32     10951.8       7281.55       63636.04     
14  1.46     0.29     13436.81      9045

In [16]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','LPG', 'CO', 'CH4'))

for idx in range(len(projection['ratio'])):
    r = projection['ratio'][idx]
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),3),
               round(PPM(r, projection['co']['m'], projection['co']['b']),3),
               round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),3),
              ))
    

    Volt     Ratio    LPG           CO            CH4          
0   0.25     9.77     0.013         0.003         0.01         
1   0.35     6.83     0.088         0.021         0.143        
2   0.45     5.2      0.38          0.091         1.103        
3   0.55     4.16     1.255         0.306         5.833        
4   0.65     3.44     3.47          0.86          24.083       
5   0.75     2.92     8.457         2.126         83.469       
6   0.85     2.51     18.783        4.779         254.055      
7   0.95     2.19     38.836        9.992         561.751      
8   1.05     1.94     75.892        19.727        776.464      
9   1.15     1.72     141.727       37.194        1081.194     
10  1.25     1.54     255.05        67.54         1487.969     
11  1.35     1.39     445.161       118.896       1932.83      
12  1.45     1.26     596.949       203.962       2495.849     
13  1.55     1.15     742.142       342.409       3174.375     
14  1.65     1.04     909.749       526.

In [17]:
## projectionRo - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']   
projectionRo['ro'] = np.arange(sensor1['ro'] - sensor1['ro']/10 , sensor1['ro'] + sensor1['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt', 'Rs', 'Ro', 'Ratio','LPG', 'CO', 'CH4'))

for idx in range(len(projectionRo['ratio'])):
    r = projectionRo['ratio'][idx]
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])

    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])

    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
           .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
              ))
    

    Volt     Rs       Ro       Ratio    LPG      CO       CH4     
0   0.06     823.33   75.81    10.86    0.01     0.0      0.0     
1   0.06     823.33   75.91    10.85    0.01     0.0      0.0     
2   0.06     823.33   76.01    10.83    0.01     0.0      0.0     
3   0.06     823.33   76.11    10.82    0.01     0.0      0.0     
4   0.06     823.33   76.21    10.8     0.01     0.0      0.0     
5   0.06     823.33   76.31    10.79    0.01     0.0      0.0     
6   0.06     823.33   76.41    10.77    0.01     0.0      0.0     
7   0.06     823.33   76.51    10.76    0.01     0.0      0.0     
8   0.06     823.33   76.61    10.75    0.01     0.0      0.0     
9   0.06     823.33   76.71    10.73    0.01     0.0      0.0     
10  0.06     823.33   76.81    10.72    0.01     0.0      0.0     
11  0.06     823.33   76.91    10.7     0.01     0.0      0.01    
12  0.06     823.33   77.01    10.69    0.01     0.0      0.01    
13  0.06     823.33   77.11    10.68    0.01     0.0      0.01

In [18]:
## projectionRo - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
Vrl    = sensor2['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor2['rs']   
projectionRo['ro'] = np.arange(sensor2['ro'] - sensor2['ro']/10 , sensor2['ro'] + sensor2['ro']/10 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt', 'Rs', 'Ro', 'Ratio','LPG', 'CO', 'CH4'))

for idx in range(len(projectionRo['ratio'])):
    r = projectionRo['ratio'][idx]
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])

    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])

    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
           .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),3),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),3),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),3),
              ))
    

    Volt     Rs       Ro       Ratio    LPG      CO       CH4     
0   0.25     190.0    17.5     10.86    0.007    0.002    0.005   
1   0.25     190.0    17.6     10.8     0.008    0.002    0.005   
2   0.25     190.0    17.7     10.74    0.008    0.002    0.005   
3   0.25     190.0    17.8     10.68    0.008    0.002    0.005   
4   0.25     190.0    17.9     10.62    0.008    0.002    0.005   
5   0.25     190.0    18.0     10.56    0.009    0.002    0.006   
6   0.25     190.0    18.1     10.5     0.009    0.002    0.006   
7   0.25     190.0    18.2     10.44    0.009    0.002    0.006   
8   0.25     190.0    18.3     10.39    0.009    0.002    0.006   
9   0.25     190.0    18.4     10.33    0.01     0.002    0.007   
10  0.25     190.0    18.5     10.27    0.01     0.002    0.007   
11  0.25     190.0    18.6     10.22    0.01     0.002    0.007   
12  0.25     190.0    18.7     10.16    0.011    0.002    0.007   
13  0.25     190.0    18.8     10.11    0.011    0.002    0.00